## Sara the Turtle: `Pandas` Redux
Recalling the script we wrote earlier to process the ARGOS tracking data for Sara the Turtle. Here, we'll demonstrate how another 3rd party package, namely `pandas`, can greatly simplify that task.

In [ ]:
#Set the user date (can be switched to be a user input...)
userDate = '7/3/2003'

In [ ]:
#Set the location classes used to filter the data. Again, can be an input
locClasses = '1','2','3'

In [ ]:
#import the modules
import pandas as pd

In [ ]:
#set a variable to the path where the tracking data lives
dataFilename = '..\Data\Sara.txt'

In [ ]:
#Open the data as a pandas dataframe. 
df = pd.read_csv(dataFilename,
                 comment='#',   #Skip lines that start with '#'
                 delimiter='\t' #Set the delimiter as <tab>
                )

In [ ]:
#Create a mask of user dates
dateMask = df['utc'].str.startswith(userDate)

In [ ]:
#Create a mask of location classe
lcMask = df['lc'].isin(locClasses)

In [ ]:
#Filter the records that match the above masks
dfOutput = df[dateMask & lcMask]
dfOutput.head()

In [ ]:
print ("On {}, Sara the turtle was found at:".format(userDate))
for i,row in dfOutput.iterrows():
    print('  {0:.3f}° Lat; {1:.3f}° Lon'.format(row['lat1'],row['lon1']))

## Mapping the points with the ArcGIS Python API

In [ ]:
#Create the GIS object
from arcgis import GIS, geometry
gis = GIS()

In [ ]:
#Create the map object
m = gis.map()

#Loop through selected records
for i,row in df[dateMask].iterrows():
    
    #Create a point, projected to WGS84
    thePoint = geometry.Point({"x":row['lon1'],
                               "y":row['lat1'],
                               "spatialRefernce":{'wkid':'4326'}
                              })
    
    #Draw that point in the map, assigning a popup to that point
    m.draw(thePoint,popup={'title':str(i),
                           'content':row['utc']})
    
    #Center the map on the current point
    m.center = [row['lat1'],row['lon1']]
    
#Display the map at zoom level 7, using oceans basemap
m.zoom = 7
m.basemap='oceans'
m